Purposes:
- speed test to find bottlenecks and further opportunities for optimization
- apply the lessons from Mike Muller's 2019 PyCon talk: https://www.youtube.com/watch?v=EcGWDNlGTNg
    

In [1]:
import cProfile
import os
import sys
import time
import timeit

In [2]:
from candidates import current_hamiltonian, current_simsignals, candidate_hamiltonian, candidate_simsignals

Import type:  <class 'numpy.ndarray'>


In [3]:
from simulation_data import spin11

In [4]:
def old_h():
    return current_hamiltonian(*spin11())

def new_h():
    return candidate_hamiltonian(*spin11())

In [5]:
old11_h = old_h()
new11_h = new_h()

In [6]:
def old_s():
    return current_simsignals(old11_h, 11)

def new_s():
    return candidate_simsignals(new11_h, 11)

Use %timeit for a 1-liner, and %%timeit for multiple lines

In [7]:
%%timeit
old_h()

1.2 s ± 25.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%%timeit
new_h()

187 ms ± 1.78 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [9]:
%%timeit
old_s()

18.6 s ± 58.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [10]:
%%timeit
new_s()

3.28 s ± 149 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
profiler = cProfile.Profile()

In [12]:
profiler.runcall(old_h)

matrix([[ 5681.875,     0.   ,     0.   , ...,     0.   ,     0.   ,
             0.   ],
        [    0.   ,  3389.125,     0.   , ...,     0.   ,     0.   ,
             0.   ],
        [    0.   ,     0.   ,  4677.125, ...,     0.   ,     0.   ,
             0.   ],
        ...,
        [    0.   ,     0.   ,     0.   , ..., -4670.875,     0.   ,
             0.   ],
        [    0.   ,     0.   ,     0.   , ...,     0.   , -3390.875,
             0.   ],
        [    0.   ,     0.   ,     0.   , ...,     0.   ,     0.   ,
         -5666.125]])

In [13]:
profiler.print_stats()

         29614 function calls (29599 primitive calls) in 1.217 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:1009(_handle_fromlist)
        1    0.003    0.003    1.217    1.217 <ipython-input-4-9838e45a9e8b>:1(old_h)
       41    0.000    0.000    0.000    0.000 <string>:1(__new__)
      506    0.000    0.000    0.000    0.000 _util.py:129(_prune_array)
      132    0.000    0.000    0.000    0.000 abc.py:137(__instancecheck__)
        2    0.000    0.000    0.000    0.000 ast.py:30(parse)
        2    0.000    0.000    0.000    0.000 ast.py:38(literal_eval)
     17/2    0.000    0.000    0.000    0.000 ast.py:64(_convert)
      385    0.000    0.000    0.000    0.000 base.py:1190(isspmatrix)
     1012    0.000    0.000    0.001    0.000 base.py:243(nnz)
      132    0.001    0.000    0.837    0.006 base.py:407(__add__)
      132    0.000    0.00

In [14]:
profiler2 = cProfile.Profile()
profiler2.runcall(new_h)
profiler2.print_stats()

         6803 function calls (6736 primitive calls) in 0.187 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       20    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:1009(_handle_fromlist)
       12    0.000    0.000    0.000    0.000 <frozen importlib._bootstrap>:416(parent)
        1    0.001    0.001    0.187    0.187 <ipython-input-4-9838e45a9e8b>:4(new_h)
      158    0.000    0.000    0.000    0.000 <string>:1(__new__)
        4    0.000    0.000    0.000    0.000 _dtype.py:319(_name_get)
        2    0.000    0.000    0.000    0.000 _methods.py:45(_all)
       55    0.000    0.000    0.000    0.000 abc.py:137(__instancecheck__)
       30    0.000    0.000    0.000    0.000 abc.py:141(__subclasscheck__)
        4    0.000    0.000    0.000    0.000 arraysetops.py:138(_unpack_tuple)
        4    0.000    0.000    0.000    0.000 arraysetops.py:151(unique)
        4    0.000    0.000    0.000    0.000 

You can save your result, then view them later with pstats.

In [15]:
cProfile.run('old_h()', 'old_h.stats')
cProfile.run('new_h()', 'new_h.stats')

In [16]:
cProfile.run('old_s()', 'old_s.stats')
cProfile.run('new_s()', 'new_s.stats')

Update 2018-05-18: vectorized_simsignals much improved! e.g. of 3.49 s on spin 11, 3.39s spent on intensity_and_energy (the calculation of I and E) and only 0.083s on the conversion to a spectrum!
Right now, Hamiltonian is not the bottleneck, and is as fast as it's going to get (for now).
In the new simsignals, the eigh is definitely the bottleneck (e.g. 2.8 out of 4.5 s) so probably can't be improved on much. However, of that ~4.5 s, ~0.86 s occurs within simsignals, so presumably in the loop. Can this be vectorized?

In [17]:
import pstats

In [18]:
stats = pstats.Stats('new_s.stats')

In [19]:
stats.print_callees('dot')

   Random listing order was used
   List reduced from 204 to 6 due to restriction <'dot'>

Function                                                                                             called...
                                                                                                         ncalls  tottime  cumtime
C:\Users\Geoffrey\Miniconda3\envs\nmr\lib\site-packages\sparse\coo\core.py:1397(dot)                 ->       1    0.000    0.153  C:\Users\Geoffrey\Miniconda3\envs\nmr\lib\site-packages\sparse\coo\common.py:234(dot)
C:\Users\Geoffrey\Miniconda3\envs\nmr\lib\site-packages\sparse\coo\common.py:59(tensordot)           ->       2    0.000    0.000  C:\Users\Geoffrey\Miniconda3\envs\nmr\lib\site-packages\scipy\sparse\base.py:1190(isspmatrix)
                                                                                                              1    0.000    0.000  C:\Users\Geoffrey\Miniconda3\envs\nmr\lib\site-packages\sparse\coo\common.py:135(<listcomp>)
  

Easier in jupyter to do this with prun.
-l 12 limits to 12 lines

In [15]:
%%prun -l 12
new_s()

         2578 function calls (2548 primitive calls) in 3.700 seconds

   Ordered by: internal time
   List reduced from 204 to 12 due to restriction <12>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    3.247    3.247    3.247    3.247 linalg.py:1324(eigh)
        1    0.173    0.173    0.173    0.173 {method 'dot' of 'numpy.ndarray' objects}
        1    0.158    0.158    0.158    0.158 common.py:1273(_dot_coo_ndarray)
        1    0.039    0.039    0.086    0.086 test_simsignals.py:243(new_compile_spectrum)
        2    0.027    0.014    0.027    0.014 {built-in method numpy.where}
        1    0.018    0.018    3.697    3.697 test_simsignals.py:271(vectorized_simsignals)
        1    0.011    0.011    0.011    0.011 {built-in method numpy.concatenate}
        1    0.011    0.011    3.593    3.593 test_simsignals.py:209(intensity_and_energy)
        2    0.009    0.004    0.009    0.004 {method 'outer' of 'numpy.ufunc' objects}
        1    0.003 

In [26]:
stats_new_s = %prun -r new_s()  # -r returns the pstats object

         2578 function calls (2548 primitive calls) in 3.389 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    2.915    2.915    2.915    2.915 linalg.py:1324(eigh)
        1    0.188    0.188    0.188    0.188 {method 'dot' of 'numpy.ndarray' objects}
        1    0.143    0.143    0.143    0.143 common.py:1273(_dot_coo_ndarray)
        1    0.037    0.037    0.083    0.083 test_simsignals.py:243(new_compile_spectrum)
        2    0.026    0.013    0.026    0.013 {built-in method numpy.where}
        1    0.021    0.021    0.021    0.021 {built-in method io.open}
        1    0.018    0.018    3.387    3.387 test_simsignals.py:271(vectorized_simsignals)
        1    0.012    0.012    3.286    3.286 test_simsignals.py:209(intensity_and_energy)
        1    0.011    0.011    0.011    0.011 {built-in method numpy.concatenate}
        2    0.009    0.004    0.009    0.004 {method 'outer' of 'numpy.ufunc' objects}
  

In [21]:
stats_new_s.print_stats()

         2578 function calls (2548 primitive calls) in 3.220 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    2.766    2.766    2.767    2.767 linalg.py:1324(eigh)
        1    0.165    0.165    0.165    0.165 {method 'dot' of 'numpy.ndarray' objects}
        1    0.160    0.160    0.160    0.160 common.py:1273(_dot_coo_ndarray)
        1    0.036    0.036    0.078    0.078 test_simsignals.py:243(new_compile_spectrum)
        2    0.025    0.012    0.025    0.012 {built-in method numpy.where}
        1    0.018    0.018    3.218    3.218 test_simsignals.py:271(vectorized_simsignals)
        1    0.011    0.011    0.011    0.011 {built-in method io.open}
        1    0.011    0.011    3.122    3.122 test_simsignals.py:209(intensity_and_energy)
        1    0.009    0.009    0.009    0.009 {built-in method numpy.concatenate}
        2    0.008    0.004    0.008    0.004 {method 'outer' of 'numpy.ufunc' objects}
  

In [27]:
%prun -T stats_new_s.txt new_s()  # -T saves results to file

 
*** Profile printout saved to text file 'stats_new_s.txt'. 


         2578 function calls (2548 primitive calls) in 3.230 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    2.796    2.796    2.796    2.796 linalg.py:1324(eigh)
        1    0.165    0.165    0.165    0.165 {method 'dot' of 'numpy.ndarray' objects}
        1    0.153    0.153    0.153    0.153 common.py:1273(_dot_coo_ndarray)
        1    0.036    0.036    0.081    0.081 test_simsignals.py:243(new_compile_spectrum)
        2    0.026    0.013    0.026    0.013 {built-in method numpy.where}
        1    0.018    0.018    3.228    3.228 test_simsignals.py:271(vectorized_simsignals)
        1    0.011    0.011    3.129    3.129 test_simsignals.py:209(intensity_and_energy)
        1    0.010    0.010    0.010    0.010 {built-in method numpy.concatenate}
        2    0.008    0.004    0.008    0.004 {method 'outer' of 'numpy.ufunc' objects}
        1    0.002    0.002    3.230    3.230 <ipython-input-6-f8a1f1193dd

In [23]:
%less stats_new_s.txt

         2578 function calls (2548 primitive calls) in 3.371 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    2.940    2.940    2.940    2.940 linalg.py:1324(eigh)
        1    0.162    0.162    0.162    0.162 {method 'dot' of 'numpy.ndarray' objects}
        1    0.157    0.157    0.157    0.157 common.py:1273(_dot_coo_ndarray)
        1    0.035    0.035    0.077    0.077 test_simsignals.py:243(new_compile_spectrum)
        2    0.025    0.013    0.025    0.013 {built-in method numpy.where}
        1    0.017    0.017    3.368    3.368 test_simsignals.py:271(vectorized_simsignals)
        1    0.011    0.011    3.274    3.274 test_simsignals.py:209(intensity_and_energy)
        1    0.009    0.009    0.009    0.009 {built-in method numpy.concatenate}
        2    0.008    0.004    0.008    0.004 {method 'outer' of 'numpy.ufunc' objects}
        1    0.002    0.002    3.371    3.371 <ipython-input-6-f8a1f1193dd

In [28]:
%prun -D stats_new_s.stats new_s()  # saves as binary instead (I think)

 
*** Profile stats marshalled to file 'stats_new_s.stats'. 


         2578 function calls (2548 primitive calls) in 3.440 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    2.998    2.998    2.998    2.998 linalg.py:1324(eigh)
        1    0.173    0.173    0.173    0.173 {method 'dot' of 'numpy.ndarray' objects}
        1    0.144    0.144    0.144    0.144 common.py:1273(_dot_coo_ndarray)
        1    0.039    0.039    0.091    0.091 test_simsignals.py:243(new_compile_spectrum)
        2    0.027    0.013    0.027    0.013 {built-in method numpy.where}
        1    0.018    0.018    3.437    3.437 test_simsignals.py:271(vectorized_simsignals)
        1    0.017    0.017    0.017    0.017 {built-in method numpy.concatenate}
        1    0.011    0.011    3.329    3.329 test_simsignals.py:209(intensity_and_energy)
        2    0.008    0.004    0.008    0.004 {method 'outer' of 'numpy.ufunc' objects}
        1    0.003    0.003    3.440    3.440 <ipython-input-6-f8a1f1193dd

In [49]:
def time_new_s():
    start_os_time0 = os.times()[0]
    start_time_clock = time.clock()
    start_default_timer = timeit.default_timer()
    start_perf = time.perf_counter()
    start_processtime = time.process_time()
    new_s()
    duration_os_time0 = os.times()[0] - start_os_time0
    duration_time_clock = time.clock() - start_time_clock
    duration_default_timer = timeit.default_timer() - start_default_timer
    duration_perf = time.perf_counter() - start_perf
    duration_processtime = time.process_time() - start_processtime
    print('os: ', duration_os_time0)
    print('time_clock: ', duration_time_clock)
    print('default_timer: ', duration_default_timer)
    print('perf: ', duration_perf)
    print('processtime: ', duration_processtime)

In [50]:
time_new_s()

C:\Users\Geoffrey\Miniconda3\envs\nmr\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  This is separate from the ipykernel package so we can avoid doing imports until


os:  11.046875
time_clock:  4.962005800000043
default_timer:  4.962005699999281
perf:  4.962005699999281
processtime:  12.0625


C:\Users\Geoffrey\Miniconda3\envs\nmr\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  if __name__ == '__main__':


Muller's recommendation is to use default_timer, which abstracts OS differences away. Very different behavior btwen Windows and Mac.

In [21]:
%load_ext snakeviz

In [22]:
time.process_time?

Docstring:
process_time() -> float

Process time for profiling: sum of the kernel and user-space CPU time.
Type:      builtin_function_or_method


In [23]:
%snakeviz new_s()

 
*** Profile stats marshalled to file 'C:\\Users\\Geoffrey\\AppData\\Local\\Temp\\tmp7zb62yv7'. 


In [53]:
# %load candidates.py
"""Collection of the current best candidates for nmrtools functions,
for testing speed etc.

"""


from nmrtools.nmrmath import hamiltonian as current_hamiltonian
from nmrtools.nmrmath import simsignals as current_simsignals

from speedtest.compare_hamiltonians import hamiltonian_sparse as \
    candidate_hamiltonian
from tests.test_simsignals import newer_simsignals as candidate_simsignals


if __name__ == '__main__':
    import numpy as np
    from simulation_data import rioux
    current_h = current_hamiltonian(*rioux())
    current_spectrum = current_simsignals(current_h, 3)
    candidate_h = candidate_hamiltonian(*rioux())
    candidate_spectrum = candidate_simsignals(candidate_h, 3)

    print(current_spectrum[:10])
    print(candidate_spectrum[:10])
    assert np.allclose(current_spectrum, candidate_spectrum)


In [24]:
%load_ext line_profiler

In [25]:
%lprun -f candidate_simsignals candidate_simsignals(new11_h, 11)

Timer unit: 1e-07 s

Total time: 3.43871 s
File: E:\Geoffrey\Documents\GitHub\cythontest\tests\test_simsignals.py
Function: vectorized_simsignals at line 271

Line #      Hits         Time  Per Hit   % Time  Line Contents
   271                                           def vectorized_simsignals(H, nspins):
   272         1   33326696.0 33326696.0     96.9      I, E = intensity_and_energy(H, nspins)
   273         1    1060373.0 1060373.0      3.1      return new_compile_spectrum(I, E)

Update 2018-05-18: new vectorization reduces new_compile_spectrum (vectorization of loop) to only 3% of time! Eigh is the clear bottleneck.
kernprof indicates that eigen is bottleneck, but the for loop is not insignificant. Perhaps this can be vectorized at some point?

In [26]:
from tests.test_simsignals import intensity_and_energy

In [27]:
%lprun -f intensity_and_energy candidate_simsignals(new11_h, 11)

Timer unit: 1e-07 s

Total time: 3.34094 s
File: E:\Geoffrey\Documents\GitHub\cythontest\tests\test_simsignals.py
Function: intensity_and_energy at line 209

Line #      Hits         Time  Per Hit   % Time  Line Contents
   209                                           def intensity_and_energy(H, nspins):
   210                                               """Calculate intensity matrix and energies (eigenvalues) from Hamiltonian.
   211                                           
   212                                               Parameters
   213                                               ----------
   214                                               H (numpy.ndarray): Spin Hamiltonian
   215                                               nspins: number of spins in spin system
   216                                           
   217                                               Returns
   218                                               -------
   219                            

In intensity_and_energy, eigh is 90% of the time, and the matrix multiplication 9.7%.